In [1]:
# !pip install torch
# !pip install tqdm
# !pip install transformers
# !pip install scikit-learn
# !pip install pandas
# !pip install wandb
# !pip install evaluate
# !pip install transformers[torch]

In [2]:
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoConfig, EarlyStoppingCallback
import evaluate
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from customModel import (customBertForSequenceClassification, customRobertaForSequenceClassification,
                         customGPT2ForSequenceClassification, customElectraForSequenceClassification,bigModelCustomRobertaForSequenceClassification,
                         addLayerCustomRobertaForSequenceClassification)
from CustomTraniner import CustomTrainer, CustomTrainer_cross_entropy, CustomTrainer_add_loss, BingModelCustomTrainer
from transformers.configuration_utils import PretrainedConfig
import wandb
import random
from transformers.models.auto.modeling_auto import MODEL_FOR_SEQUENCE_CLASSIFICATION_MAPPING
from frontModelCustom import frontModelDataset, data_labels



/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/leesk/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
def set_seed(seed:int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
set_seed(42)

### Data Process

In [5]:
# data = pd.read_csv('./genre_11_tempo_4.csv')
# data = pd.read_csv('./not_duplicate_data.csv')
# data = pd.read_csv('./data_llama2.csv')
data = pd.read_csv('./total_data_no_world.csv')
# data.head()

In [6]:
with open('labels.pkl','wb') as f:
    pickle.dump(data.emotion.unique(),f)
    pickle.dump(data['tempo(category)'].unique(),f)
    pickle.dump(data['genre'].unique(),f)

In [7]:
emotion , tempo, genre = data_labels('labels.pkl')

In [8]:
BASE_MODEL = 'FacebookAI/roberta-large'

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
config = AutoConfig.from_pretrained(BASE_MODEL)
config.num_labels1 = len(emotion)
config.num_labels2 = len(tempo)
config.num_labels3 = len(genre)

# model = customBertForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)
model = customRobertaForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)
# model = addLayerCustomRobertaForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)

# model = bigModelCustomRobertaForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)
# model = customElectraForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)
# model = customGPT2ForSequenceClassification.from_pretrained(BASE_MODEL, config= config).to(device)
# tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = model.config.eos_token_id

Some weights of customRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier1.bias', 'classifier1.weight', 'classifier2.bias', 'classifier2.weight', 'classifier3.bias', 'classifier3.weight', 'dense1.bias', 'dense1.weight', 'dense2.bias', 'dense2.weight', 'dense3.bias', 'dense3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
type(config)

transformers.models.roberta.configuration_roberta.RobertaConfig

In [10]:
data2 = data.copy()
# emotion_data = data2.groupby('emotion').sample(frac=0.05, random_state=42)
# tempo_data = data2.groupby('tempo(category)').sample(frac=0.05, random_state=42)
# genre_data = data2.groupby('genre').sample(frac=0.05, random_state=42)
# index_total = set(emotion_data.index) | set(tempo_data.index) | set(genre_data.index)
# valid_data = data2.iloc[list(index_total)]
# train_data = data2.drop(list(index_total)).sample(frac=1, random_state=42)
data_valid_index = data2.groupby(['emotion','genre','tempo(category)']).sample(frac=0.1, random_state=42).index
valid_data = data2.iloc[data_valid_index]
train_data = data2.drop(list(data_valid_index)).sample(frac=1, random_state=42)


In [11]:
# dataset_train = frontModelDataset(train_data, tokenizer =tokenizer)
# dataset_valid = frontModelDataset(valid_data, tokenizer =tokenizer)

dataset_train = frontModelDataset(train_data, tokenizer =tokenizer)
dataset_valid = frontModelDataset(valid_data, tokenizer =tokenizer)



69717it [00:05, 13244.01it/s]
7758it [00:00, 13848.13it/s]


In [12]:
# datas = []
# for idx, df in tqdm(train_data.iterrows()):
#     if pd.notnull(df.caption) and isinstance(df.caption, str):
#         datas.append(df.caption)
#     else:
#         print(df.caption)
# tokenizer(datas,padding=True, truncation=True,max_length=512 ,return_tensors="pt").to('cuda')

In [13]:

def get_preds_from_logits(logits):
    ret = np.zeros(logits.shape)
    
    # The first 5 columns (GLOBAL_SCORE_INDICES) are for global scores. They should be handled with a multiclass approach
    # i.e. we fill 1 to the class with highest probability, and 0 into the other columns
    best_class = np.argmax(logits, axis=1)
    ret[list(range(len(ret))), best_class] = 1
    # The other columns are for causes and emotions. They should be handled with multilabel approach.
    # i.e. we fill 1 to every class whose score is higher than some threshold
    # In this example, we choose that threshold = 0
    
    return ret

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    final_metrics = {}
    
    # Deduce predictions from logits
    predictions_emotion = get_preds_from_logits(logits[0])
    predictions_tempo = get_preds_from_logits(logits[1])
    predictions_genre = get_preds_from_logits(logits[2])
    
    # Get f1 metrics for global scoring. Notice that f1_micro = accuracy
    final_metrics["f1_emotion"] = f1_score(labels[0], predictions_emotion, average="micro")
    
    # Get f1 metrics for causes
    final_metrics["f1_tempo"] = f1_score(labels[1], predictions_tempo, average="micro")
    

    # The global f1_metrics
    final_metrics["f1_genre"] = f1_score(labels[2], predictions_genre, average="micro")

    final_metrics['fi_total'] = (final_metrics["f1_emotion"] + final_metrics["f1_tempo"] + final_metrics["f1_genre"])/3
    
    # Classification report
    # print("Classification report for global scores: ")
    # print(classification_report(labels[:, GLOBAL_SCORE_INDICES], predictions[:, GLOBAL_SCORE_INDICES], zero_division=0))
    # print("Classification report for causes: ")
    # print(classification_report(labels[:, CAUSE_INDICES], predictions[:, CAUSE_INDICES], zero_division=0))
    return final_metrics

In [15]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
print(config.num_labels1, config.num_labels2, config.num_labels3)

wandb.init(project="Final project", entity="sanggang",name = "no_duplicate_"+BASE_MODEL)

training_args = TrainingArguments(

   output_dir="my_awesome_model",
   save_steps=300,
   eval_steps = 300, 
   warmup_steps=500,
   logging_steps=100,
   learning_rate=5e-5,
   per_device_train_batch_size=32,
   per_device_eval_batch_size=32,
   num_train_epochs=6,
   weight_decay=0.01,
   evaluation_strategy='steps',
   load_best_model_at_end = True,
   save_total_limit = 2,
   report_to="wandb",
   metric_for_best_model='fi_total',
   # run_name=BASE_MODEL, 
)

# trainer = CustomTrainer(

#    model=model,
#    args=training_args,
#    train_dataset=dataset_train,
#    eval_dataset=dataset_valid,
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics,
# )


trainer = CustomTrainer_add_loss(

   model=model,
   args=training_args,
   train_dataset=dataset_train,
   eval_dataset=dataset_valid,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
   callbacks = [EarlyStoppingCallback(early_stopping_patience=5)],
)

# trainer = BingModelCustomTrainer(

#    model=model,
#    args=training_args,
#    train_dataset=dataset_train,
#    eval_dataset=dataset_valid,
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics,
# )

# trainer = CustomTrainer_cross_entropy(
#    model=model,
#    args=training_args,
#    train_dataset=dataset_train,
#    eval_dataset=dataset_valid,
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics,
# )

# trainer = Trainer(

#    model=model,
#    args=training_args,
#    train_dataset=dataset_train,
#    eval_dataset=dataset_valid,
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics,
# )



trainer.train()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


10 4 11


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: leesk9663 (sanggang). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 